# this notebook parse patch data
# TODO: meet with Venu to confirm the changes I made
# TODO: figure out the units in COSMED

In [2]:
import numpy as np
import argparse

import os
import math
from math import sin

import json

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torchsummary import summary
torch.manual_seed(0)

i_seed = 0

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from plotting_tools import *
from setting import *
# from models import *
# from models_CNN import *
from evaluate import *

from stage3_preprocess import *
from stage4_regression import *
# from training_util import *
# from dataset_util import *
# from FL_extension.training_util import *
from FL_extension.dataset_util import *
from FL_extension.evaluation_util import *
# from FL_extension.models import *
# from FL_extension.models_CNNlight import *
from dataIO import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [3]:
print(torch.version.cuda)

10.2


In [4]:
parser = argparse.ArgumentParser(description='SpO2_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')


# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/stage3/', 
                          '--output_folder', '../../data/stage4/MET_regression/',
                          '--training_params_file', 'training_params_baseline.json',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage3/', output_folder='../../data/stage4/MET_regression/', training_params_file='training_params_baseline.json')


In [5]:
inputdir = args.input_folder
outputdir = args.output_folder
training_params_file = args.training_params_file

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

    

# get training params and dataloaders

In [6]:
torch.cuda.is_available()

True

In [7]:
with open(training_params_file) as json_file:
    training_params_list = json.load(json_file)

for training_params in training_params_list:
    # include device in training_params
#     device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
#     training_params['device'] = device


    training_params['CV_config'] = {
        'subject_id': 113,
        'task_id': 1,
    }
    stage3_dict = data_loader('stage3_dict', inputdir).item()
    training_params['list_signal'] = stage3_dict['list_signal']
    training_params['list_feature'] = stage3_dict['list_feature']
    training_params['list_output'] = stage3_dict['list_output']
    training_params['list_meta'] = stage3_dict['list_meta']
    training_params['FS_RESAMPLE_DL'] = stage3_dict['FS_RESAMPLE_DL']
    training_params['subject_ids'] = stage3_dict['subject_ids']
    training_params['task_ids'] = stage3_dict['task_ids']
    
#     input_CV = '../../data/stage3/113/CV2/'
#     dataloaders, dataset_sizes = get_loaders(input_CV, training_params)
    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    print('data dimensions are:', dataloaders['train'].dataset.data.shape)

    data_dimensions = dataloaders['train'].dataset.__getitem__(0)[0].size()
    training_params['data_dimensions'] = list(data_dimensions)
    
    sweep_name = training_params['sweep_name'] 
    

training_params = training_params_list[0]

FileNotFoundError: [Errno 2] No such file or directory: 'training_params_baseline.json'

In [ ]:

training_params['CV_config'] = {
    'subject_id': 113,
    'task_id': 1,
}



In [8]:
training_params['output_names']

NameError: name 'training_params' is not defined

# compute MET using our own data

## sedentary MET=1.5 kcal/kg/h = 1.5/60 kcal/kg/min = 0.025 kcal/kg/min
### ref: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4448542/

In [ ]:
# # dataloaders['train'].dataset.data

# i_feature = training_params['feature_names'].index('weight')
# i_label = training_params['output_names'].index('EE_cosmed')


# MET_train = dataloaders['train'].dataset.label[:,i_label] / dataloaders['train'].dataset.feature[:,i_feature]
# MET_train = MET_train.mean()
# print('MET_train is {:.3f} kcal/kg/min'.format(MET_train))

In [ ]:

# def get_df_performance(df_performance, label, label_est, task, subject_id):
#     rmse = np.sqrt(mean_squared_error(label, label_est))

#     mae, _ = get_MAE(label, label_est)
#     mape, _ = get_MAPE(label, label_est)

#     Rsquared = get_CoeffDeterm(label=label, predictions=label_est)

#     df_performance.append(pd.DataFrame({
#         'CV': [subject_id],
#         'task': [task],
#         'Rsquared': [Rsquared],
#         'rmse': [rmse],
#         'mae': [mae],
#         'mape': [mape],
#     }))
    
#     return df_performance

In [ ]:
task = training_params['output_names'][0]

In [ ]:

# def get_df_performance(df_performance, label, label_est, task, subject_id):
#     rmse = np.sqrt(mean_squared_error(label, label_est))

#     mae, _ = get_MAE(label, label_est)
#     mape, _ = get_MAPE(label, label_est)

#     Rsquared = get_CoeffDeterm(label=label, predictions=label_est)

#     df_performance.append(pd.DataFrame({
#         'CV': [subject_id],
#         'task': [task],
#         'Rsquared': [Rsquared],
#         'rmse': [rmse],
#         'mae': [mae],
#         'mape': [mape],
#     }))
    
#     return df_performance

In [ ]:


#     ordered_subject_ids = np.asarray([115, 107, 113, 110, 101, 104, 106, 121, 212, 102, 103, 111, 114, 116, 118, 119, 120])



#     # for subject_id in training_params['subject_ids']:
#     for subject_id in ordered_subject_ids:

#         training_params['CV_config']['subject_id'] = subject_id

#         device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
#         print('using device', device)


#         print('using model ', training_params['model_name'])

#         model = resp_multiverse(training_params=training_params)
# #         print(model)
        
#         model = model.to(device).float()

#         optimizer = torch.optim.Adam(model.parameters(), lr=training_params['learning_rate'], weight_decay=0.01)

#         training_params['criterion'] = criterion
#         training_params['optimizer'] = optimizer
#         training_params['inputdir'] = inputdir

#         CV_dict = train_model(model, training_params, trainer, evaler, preder)

# #             sys.exit()

#         plot_losses(CV_dict, outputdir=outputdir, show_plot=False)

#         for task in training_params['tasks']:






#             label_est_val = CV_dict['performance_dict_val']['out_dict'][task]
#             label_val = CV_dict['performance_dict_val']['label_dict'][task]

#             label_est_train = CV_dict['performance_dict_train']['out_dict'][task]
#             label_train = CV_dict['performance_dict_train']['label_dict'][task]

#             # get performance df for training and testing dataset
#             df_performance_train[task] = df_performance_train[task].append( get_df_performance(label_train, label_est_train, subject_id, task), ignore_index=True )
#             df_performance_train[task].to_csv(outputdir+'df_performance_train_{}.csv'.format(task), index=False)

#             df_outputlabel_train[task] = df_outputlabel_train[task].append(
#                 pd.DataFrame( {
#                 'label_est': label_est_train,
#                 'label': label_train,
#                 'CV': [subject_id]*label_train.shape[0],
#                 'task': [task]*label_train.shape[0]
#                 }), ignore_index=True )

#             df_outputlabel_train[task].to_csv(outputdir+'df_outputlabel_train_{}.csv'.format(task), index=False)

#             df_performance_val[task] = df_performance_val[task].append( get_df_performance(label_val, label_est_val, subject_id, task), ignore_index=True )
#             df_performance_val[task].to_csv(outputdir+'df_performance_val_{}.csv'.format(task), index=False)

#             df_outputlabel_val[task] = df_outputlabel_val[task].append(
#                 pd.DataFrame( {
#                 'label_est': label_est_val,
#                 'label': label_val,
#                 'CV': [subject_id]*label_val.shape[0],
#                 'task': [task]*label_val.shape[0]
#                 }), ignore_index=True )

#             df_outputlabel_val[task].to_csv(outputdir+'df_outputlabel_val_{}.csv'.format(task), index=False)


In [ ]:
df_performance_train = {}
df_performance_val = {}

df_outputlabel_train = {}
df_outputlabel_val = {}

# for task in training_params['output_names']:
task = 'EE_cosmed'
#     if 'EE' not in task:
#         continue

df_performance_train[task] = pd.DataFrame()
df_performance_val[task] = pd.DataFrame()

df_outputlabel_train[task] = pd.DataFrame()
df_outputlabel_val[task] = pd.DataFrame()

In [ ]:
# # plt.plot(dataloaders['train'].dataset.label[:,i_label])
# plt.plot( dataloaders['train'].dataset.label[:,i_label] / dataloaders['train'].dataset.feature[:,i_feature])
# # MET_train

In [ ]:
# dataloaders['train']

In [ ]:
training_params['feature_names']

In [ ]:
# df_performance_train = []
# df_performance_val = []

# df_outputlabel_train = []
# df_outputlabel_val = []


training_params['feature_names'] = ['weight']
training_params['output_names'] = ['EE_cosmed']

for subject_id in training_params['subject_ids']:

    training_params['CV_config']['subject_id'] = subject_id

    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    
    # convert feature back to raw values
    
    feature_mean = dataloaders['feature_mean']
    feature_std = dataloaders['feature_std']

    features_train = dataloaders['train'].dataset.feature
    features_train = features_train * feature_std + feature_mean

    features_val = dataloaders['val'].dataset.feature
    features_val = features_val * feature_std + feature_mean
    
    
    i_feature = training_params['feature_names'].index('weight')
    i_label = training_params['output_names'].index('EE_cosmed')

    MET_train = dataloaders['train'].dataset.label[:,i_label] / features_train[:,i_feature]
    MET_train = MET_train.mean()

    label_est_train = MET_train*features_train[:,i_feature]
    label_train = dataloaders['train'].dataset.label[:,i_label]
    
    label_est_val = MET_train*features_val[:,i_feature]
    label_val = dataloaders['val'].dataset.label[:,i_label]
    
    
    
    
    # get performance df for training and testing dataset
    df_performance_train[task] = df_performance_train[task].append( get_df_performance(label_train, label_est_train, subject_id, task), ignore_index=True )
    df_performance_train[task].to_csv(outputdir+'df_performance_train_{}.csv'.format(task), index=False)

    df_outputlabel_train[task] = df_outputlabel_train[task].append(
        pd.DataFrame( {
        'label_est': label_est_train,
        'label': label_train,
        'CV': [subject_id]*label_train.shape[0],
        'task': [task]*label_train.shape[0]
        }), ignore_index=True )

    df_outputlabel_train[task].to_csv(outputdir+'df_outputlabel_train_{}.csv'.format(task), index=False)

    df_performance_val[task] = df_performance_val[task].append( get_df_performance(label_val, label_est_val, subject_id, task), ignore_index=True )
    df_performance_val[task].to_csv(outputdir+'df_performance_val_{}.csv'.format(task), index=False)

    df_outputlabel_val[task] = df_outputlabel_val[task].append(
        pd.DataFrame( {
        'label_est': label_est_val,
        'label': label_val,
        'CV': [subject_id]*label_val.shape[0],
        'task': [task]*label_val.shape[0]
        }), ignore_index=True )

    df_outputlabel_val[task].to_csv(outputdir+'df_outputlabel_val_{}.csv'.format(task), index=False)

            
            
#     df_performance_train = get_df_performance(label_train, label_est_train, subject_id, task)
#     df_performance_val = get_df_performance(label_val, label_est_val, subject_id, task )

        
#     df_outputlabel_val.append(
#         pd.DataFrame( {
#         'label_est': label_est_val,
#         'label': label_val,
#         'CV': subject_id,
#         'MET_train': MET_train
# #             'task': [task]*label_val.shape[0]
#         })
#     )
#     # df_outputlabel
#     df_outputlabel_train.append(
#         pd.DataFrame( {
#         'label_est': label_est_train,
#         'label': label_train,
#         'CV': subject_id,
#         'MET_train': MET_train

# #             'task': [task]*label_train.shape[0]
#         })
#     )

# df_performance_train = pd.concat(df_performance_train)
# df_performance_val = pd.concat(df_performance_val)

# df_outputlabel_train = pd.concat(df_outputlabel_train)
# df_outputlabel_val = pd.concat(df_outputlabel_val)

In [ ]:
df_outputlabel = df_outputlabel_val[task]
df_performance = df_performance_val[task]

N_sub = len(df_outputlabel['CV'].unique())
N_samples = df_outputlabel.shape[0]
t_dur = N_samples*3/60
PCC = get_PCC(df_outputlabel['label'], df_outputlabel['label_est'])
MAE = get_MAE(df_outputlabel['label'], df_outputlabel['label_est'])[0]
RMSE = get_RMSE(df_outputlabel['label'], df_outputlabel['label_est'])
MAPE = get_MAPE(df_outputlabel['label'], df_outputlabel['label_est'])[0]

Rsquared = get_CoeffDeterm(df_outputlabel['label'], df_outputlabel['label_est'])

label_range = [df_outputlabel['label'].min(), df_outputlabel['label'].max()]
task_name = task.split('_')[0]

title_str = '{} range: {:.1f}-{:.1f} {}'.format(task.split('_')[0], label_range[0], label_range[1], unit_dict[task_name])
textstr = 'RMSE={:.2f} {}\nMAE={:.2f} {}\nMAPE={:.2f} {}\nPCC={:.2f}\nR2={:.2f}\nN_sub={}\nN_samples={}\nduration={:.2f} min'.format(
    RMSE, unit_dict[task_name], MAE, unit_dict[task_name], MAPE*100, '%', PCC, Rsquared,
    N_sub, N_samples, t_dur)

print(textstr)

In [ ]:
# df_performance_val['rmse'].mean(), df_performance_val['mae'].mean(), df_performance_val['mape'].mean()

In [ ]:
# df_performance_val

In [ ]:
    
# def plot_regression_partial(ax, subjects_performance_dict, subject_id_plt, training_params,outputdir=None, show_plot=False, log_wandb=False):

#     agg_performance_dict = aggregate_performance_dict(subjects_performance_dict)

#     props = dict(boxstyle='round,pad=0.7', facecolor='white', edgecolor='black', alpha=0.7)
#     fontsize = 16

#     subject_ids = list(subjects_performance_dict.keys())
#     N_beats_val = 0


#     for subject_id in subject_ids:
#         if int(subject_id)//100 == 0:
#             marker = marker_dict['x']
#         elif int(subject_id)//100 == 1:
#             marker = marker_dict['circle']

#         performance_dict = subjects_performance_dict[subject_id]

#         SpO2_val = performance_dict['SpO2_val']
#         SpO2_est_val = performance_dict['SpO2_est_val']


#         color = color_dict[color_names[int(subject_id)%100]]

#         if subject_id == subject_id_plt:
#             alpha=0.6
#             ax.set_title('{}'.format(subject_id), fontsize=fontsize+5)
#             rmse_val = performance_dict['rmse_val']
#         else:
#             alpha=0.03

#         ax.scatter(SpO2_val, SpO2_est_val, alpha=alpha, color=color, marker=marker)

#         N_beats_val += SpO2_val.shape[0]
        
    
#     performance_dict = subjects_performance_dict[subject_id_plt]
# #     print(training_params['use_calibration'], subjects_performance_dict.keys())
#     if training_params['use_calibration']:
#         if 'SpO2_cal_range' in performance_dict:
#             SpO2_cal_range = performance_dict['SpO2_cal_range']
#             subject_id_plt = str(subject_id_plt) + '\n[' + r'$SpO_{2}$' + ' range: {:.1f}-{:.1f}%]'.format(SpO2_cal_range[0], SpO2_cal_range[1])
#     ax_conditioning2(ax, subject_id_plt, fontsize, props, 'RMSE={:.2f}%'.format(rmse_val))
    
    

In [ ]:
        
# def ax_conditioning(ax, title_str, fontsize, props, textstr):
#     ax.set_title(title_str, fontsize=fontsize+5)

#     ax.set_ylabel(r'$SpO_{2}$'+' estimated (%)', fontsize=fontsize)
#     ax.set_xlabel(r'$SpO_{2}$'+' label (%)', fontsize=fontsize)

#     major_ticks = np.arange(label_range_dict['SpO2'][0],label_range_dict['SpO2'][1]+1,5)
#     minor_ticks = np.arange(label_range_dict['SpO2'][0],label_range_dict['SpO2'][1]+1)

#     ax.set_xticks(major_ticks)
#     ax.set_xticks(minor_ticks, minor=True)
#     ax.set_yticks(major_ticks)
#     ax.set_yticks(minor_ticks, minor=True)
#     # Or if you want different settings for the grids:
#     ax.grid(which='minor', alpha=0.3)
#     ax.grid(which='major', alpha=0.8)


#     ax.plot( label_range_dict['SpO2'],label_range_dict['SpO2'] , color='gray', alpha=0.5)
# #     ax.legend(loc='upper left', frameon=True, fontsize=fontsize-7)


#     # place a text box in bottom right in axes coords
# #     ax.text(0.5, 0.08, textstr, transform=ax.transAxes, fontsize=fontsize-5,
# #     verticalalignment='bottom', horizontalalignment='left', bbox=props)
#     ax.set_ylim(label_range_dict['SpO2'])
#     ax.set_xlim(label_range_dict['SpO2'])
    
    

In [ ]:
# sns.scatterplot(data=aaa, x='label', y='label_est', hue='CV', palette=subject_palette)

In [ ]:
def ax_no_top_right(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

